In [4]:
%pip install tabulate

import tenseal as ts
import tenseal.sealapi as seal
import random
import pickle
import numpy as np
import math
from IPython.display import HTML, display
import tabulate
import pytest
 
def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])

enc_type_str = {
    ts.ENCRYPTION_TYPE.SYMMETRIC : "symmetric", 
    ts.ENCRYPTION_TYPE.ASYMMETRIC : "asymmetric",
}

scheme_str = {
    ts.SCHEME_TYPE.CKKS : "ckks", 
    ts.SCHEME_TYPE.BFV : "bfv",
}

def decrypt(enc):
    return enc.decrypt()

Looking in indexes: http://mirrors.baidubce.com/pypi/simple/
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'pytest'

In [10]:
# context.auto_rescale = True
print("Automatic relinearization is:", ("on" if context.auto_relin else "off"))
print("Automatic rescaling is:", ("on" if context.auto_rescale else "off"))
print("Automatic modulus switching is:", ("on" if context.auto_mod_switch else "off"))

Automatic relinearization is: on
Automatic rescaling is: on
Automatic modulus switching is: on


In [8]:
poly_mod = 8192
coeff_mod_bit_sizes = [60, 40, 40, 60]
enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
prec = 40

data = [ random.random()]
for data_pow in [1]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    # print("type(data):", type(data))
    # print("data:", data)
context = ts.context(
    scheme=ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=poly_mod,
    coeff_mod_bit_sizes=coeff_mod_bit_sizes,
    encryption_type=enc_type,
)
scale = 2 ** prec

print("type(data):", type(data))
print("data:", data)
ckks_vec = ts.ckks_vector(context, data, scale)

type(data): <class 'list'>
data: [2.846338549683817]


In [15]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        try:
            ckks_mul = ckks_vec * ckks_vec 
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
        
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 20]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -26
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -21
[2^1 - 2^2],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -16
[2^5 - 2^6],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -7


In [17]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        # try:
        #     ckks_mul = ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
        
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 20]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul3,decryption prec 2 ** -29
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul3,decryption prec 2 ** -18
[2^1 - 2^2],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul3,decryption prec 2 ** -9


In [19]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        # try:
        #     ckks_mul = ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
                break
        
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 20]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul5,decryption prec 2 ** -27
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul5,decryption prec 2 ** -16
[2^1 - 2^2],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul5,decryption prec 2 ** -9


In [20]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        # try:
        #     ckks_mul = ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
        
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 20]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul7,decryption prec 2 ** -28
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul7,decryption prec 2 ** -19
[2^1 - 2^2],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul7,decryption prec 2 ** -1


In [21]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        # try:
        #     ckks_mul = ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
        
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 20]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul10,decryption prec 2 ** -29
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul10,decryption prec 2 ** -20


In [22]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        # try:
        #     ckks_mul = ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
        
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
                break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 20]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul15,decryption prec 2 ** -29
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul15,decryption prec 2 ** -23


In [23]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        # try:
        #     ckks_mul = ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
        
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul18", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 19]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul18", "decryption prec 2 ** {}".format(-dec_prec)])
                break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul18,decryption prec 2 ** -31
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul18,decryption prec 2 ** -14


In [14]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (32768, [60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60], 40),
        # (32768, [60, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            print("error")
            continue
            
        try:
            ckks_mul = ckks_vec * ckks_vec 
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul1", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] **4]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul3", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
        
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec 
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 8]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul7", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
                
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 11]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul10", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                
        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 16]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul15", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 20]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul19", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break

        # try:
        #     ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        # except:
        #     ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "failed"])
        #     continue
        # decrypted = decrypt(ckks_mul)
        # for dec_prec in reversed(range(prec)):
        #     if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 21]:
        #         ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul20", "decryption prec 2 ** {}".format(-dec_prec)])
        #         break
display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -27
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul3,decryption prec 2 ** -30
[2^-5 - 2^-4],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul10,decryption prec 2 ** -29
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -19
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul3,decryption prec 2 ** -16
[2^-1 - 2^0],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul10,decryption prec 2 ** -14
[2^1 - 2^2],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -16
[2^1 - 2^2],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul3,decryption prec 2 ** -10
[2^5 - 2^6],32768,"[60, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 60]",2**40,mul1,decryption prec 2 ** -7


In [26]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]

# We add more depth for the multiplication scenario
for data_pow in [-5, -1, 1, 5, 10]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (16384, [60, 40, 40, 40, 40, 40, 40, 60], 40),
        # (16384, [60, 40, 40, 40, 40, 40, 60], 40),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            continue
            
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 6]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul5", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                    
        # 解密后再乘以一个数        
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
            ckks_vec2 = ts.ckks_vector(context, decrypted, scale)
        except BaseException as e:
            continue
        try:
            ckks_mul = ckks_vec2 * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul6*", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 7]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul6*", "decryption prec 2 ** {}".format(-dec_prec)])
                break

        # 直接相乘6次后解密
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            continue
        try:
            ckks_mul = ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul6", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] ** 7]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul6", "decryption prec 2 ** {}".format(-dec_prec)])
                break

display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-5 - 2^-4],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul5,decryption prec 2 ** -29
[2^-5 - 2^-4],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul6*,decryption prec 2 ** -31
[2^-5 - 2^-4],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul6,decryption prec 2 ** -29
[2^-1 - 2^0],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul5,decryption prec 2 ** -18
[2^-1 - 2^0],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul6*,decryption prec 2 ** -18
[2^-1 - 2^0],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul6,decryption prec 2 ** -18
[2^1 - 2^2],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul5,decryption prec 2 ** -5
[2^1 - 2^2],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul6*,decryption prec 2 ** -3
[2^1 - 2^2],16384,"[60, 40, 40, 40, 40, 40, 40, 60]",2**40,mul6,decryption prec 2 ** -3


In [42]:
data = [ random.random()]

enc_type = ts.ENCRYPTION_TYPE.ASYMMETRIC
ct_size_benchmarks = [["Value range", "Polynomial modulus", "Coefficient modulus sizes", "Precision", "Operation", "Status"]]


for data_pow in [-1, 0, 1, 5, 11, 21, 41, 51]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (8192, [60, 40, 60], 40),
        (8192, [40, 21, 21, 21, 21, 21, 21, 40], 40),
        (8192, [40, 21, 21, 21, 21, 21, 21, 40], 21),
        (8192, [40, 20, 40], 40),
        (8192, [20, 20, 20], 38),
        (8192, [60, 60], 38),
        (8192, [40, 40], 38),
        (8192, [17, 17], 15),
        (4096, [40, 20, 40], 40),
        (4096, [30, 20, 30], 40),
        (4096, [20, 20, 20], 38),
        (4096, [19, 19, 19], 35),
        (4096, [18, 18, 18], 33),
        (4096, [30, 30], 25),
        (4096, [25, 25], 20),
        (4096, [18, 18], 16),
        (4096, [17, 17], 15),
        (2048, [20, 20], 18),
        (2048, [18, 18], 16),
        (2048, [16, 16], 14),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "encrypt", "encryption failed"])
            continue
    
        decrypted = decrypt(ckks_vec)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == data:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "encrypt", "decryption prec 2 ** {}".format(-dec_prec)])
                break
        ckks_sum = ckks_vec + ckks_vec
        decrypted = decrypt(ckks_sum)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] + data[0]]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "sum", "decryption prec 2 ** {}".format(-dec_prec)])
                break
                

# We add more depth for the multiplication scenario
for data_pow in [-1, 0, 1, 5, 11, 21, 41, 51]:
    data = [ random.uniform(2 ** data_pow, 2 ** (data_pow + 1))]
    for (poly_mod, coeff_mod_bit_sizes, prec) in [
        (8192, [60, 40, 40, 60], 40),
        (8192, [40, 21, 21, 40], 40),
        (8192, [40, 21, 21, 40], 21),
        (8192, [40, 20, 20, 40], 40),
        (8192, [20, 20, 20], 38),
        (4096, [40, 20, 40], 40),
        (4096, [30, 20, 30], 40),
        (4096, [20, 20, 20], 38),
        (4096, [19, 19, 19], 35),
        (4096, [18, 18, 18], 33),
        (4096, [30, 30, 30], 25),
        (4096, [25, 25, 25], 20),
        (4096, [18, 18, 18], 16),
        (2048, [18, 18, 18], 16),
    ]:
        val_str = "[2^{} - 2^{}]".format(data_pow, data_pow + 1)
        context = ts.context(
            scheme=ts.SCHEME_TYPE.CKKS,
            poly_modulus_degree=poly_mod,
            coeff_mod_bit_sizes=coeff_mod_bit_sizes,
            encryption_type=enc_type,
        )
        scale = 2 ** prec
        try:
            ckks_vec = ts.ckks_vector(context, data, scale)
        except BaseException as e:
            continue
                
        try:
            ckks_mul = ckks_vec * ckks_vec
        except:
            ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul", "failed"])
            continue
        decrypted = decrypt(ckks_mul)
        for dec_prec in reversed(range(prec)):
            if pytest.approx(decrypted, abs=2 ** -dec_prec) == [data[0] * data[0]]:
                ct_size_benchmarks.append([val_str, poly_mod, coeff_mod_bit_sizes, "2**{}".format(prec), "mul", "decryption prec 2 ** {}".format(-dec_prec)])
                break

display(HTML(tabulate.tabulate(ct_size_benchmarks, tablefmt='html')))

Value range,Polynomial modulus,Coefficient modulus sizes,Precision,Operation,Status
[2^-1 - 2^0],8192,"[60, 40, 60]",2**40,encrypt,decryption prec 2 ** -28
[2^-1 - 2^0],8192,"[60, 40, 60]",2**40,sum,decryption prec 2 ** -27
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**40,encrypt,decryption prec 2 ** -36
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**40,sum,decryption prec 2 ** -35
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**21,encrypt,decryption prec 2 ** -13
[2^-1 - 2^0],8192,"[40, 21, 21, 21, 21, 21, 21, 40]",2**21,sum,decryption prec 2 ** -12
[2^-1 - 2^0],8192,"[40, 20, 40]",2**40,encrypt,decryption prec 2 ** -29
[2^-1 - 2^0],8192,"[40, 20, 40]",2**40,sum,decryption prec 2 ** -28
[2^-1 - 2^0],8192,"[20, 20, 20]",2**38,encrypt,decryption prec 2 ** -28
[2^-1 - 2^0],8192,"[60, 60]",2**38,encrypt,decryption prec 2 ** -26
